In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
import sys
from time import time
# from mpl_toolkits.basemap import Basemap

In [3]:
base_dir = "../dataset"
# nc.Dataset(f"{base_dir}/")
df_locations = pd.read_csv(f"{base_dir}/SKNlocations.csv")
df_data = pd.read_excel(f"{base_dir}/FilledDataset2012.xlsx", sheet_name="Data_in")

In [7]:
df_data

,SKN,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1.0,1920,4.76,0.23,0.005188,0.43,0.30,NaN,NaN,0,0,0.87,NaN,NaN
1,1.0,1921,7.78,1.29,2.946572,0.58,0.27,NaN,NaN,0.58,0,0.44,NaN,NaN
2,1.0,1922,4.02,2.75,6.053961,1.63,0.03,NaN,NaN,0.04,0.74,0.95,NaN,NaN
3,1.0,1923,11.47,2.01,3.740279,4.18,0.27,NaN,NaN,1.01,4.94,2.21,NaN,NaN
4,1.0,1924,0,0.28,2.168428,3.73,0.62,NaN,NaN,0.32,0,2.23,NaN,2.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197062,1147.0,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197063,1147.0,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197064,1147.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197065,1147.0,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_locations.head()

,SKN,Name,Lat_DD,Lon_DD
0,1.0,KALAE,18.916176,-155.674994
1,1.1,MORSE FIELD,18.913680,-155.680550
2,1.2,KALAE S TRK STA,18.938669,-155.680549
3,2.0,MANUKA,19.108660,-155.825545
4,2.1,KAHUKU MAUKA 2.10,19.108890,-155.746670


In [9]:
df_data.head()

,SKN,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1.0,1920,4.76,0.23,0.005188,0.43,0.30,NaN,NaN,0,0,0.87,NaN,NaN
1,1.0,1921,7.78,1.29,2.946572,0.58,0.27,NaN,NaN,0.58,0,0.44,NaN,NaN
2,1.0,1922,4.02,2.75,6.053961,1.63,0.03,NaN,NaN,0.04,0.74,0.95,NaN,NaN
3,1.0,1923,11.47,2.01,3.740279,4.18,0.27,NaN,NaN,1.01,4.94,2.21,NaN,NaN
4,1.0,1924,0,0.28,2.168428,3.73,0.62,NaN,NaN,0.32,0,2.23,NaN,2.48


## Conevrt df_data into cell-based matrix

In [10]:
X = []
for index, row in df_data.iterrows():
    if row.Year < 1948:
        # No need to keep data older than 1948 becase no data exists in netCDF files
        continue
    for i, cell in enumerate(row[2:]):
        X.append([row.SKN, row.Year, i + 1, cell])

In [11]:
df_data_by_cell = pd.DataFrame(X, columns = ["skn", "year", "month", "data_in"]).dropna()
df_data_by_cell = df_data_by_cell.replace(r'^\s*$', np.nan, regex=True).dropna()



In [12]:
df_data_by_cell.head()

,skn,year,month,data_in
0,1.0,1948,1,3.2
1,1.0,1948,2,1.33
2,1.0,1948,3,2.31
3,1.0,1948,4,3.23
4,1.0,1948,5,1.14


In [13]:
# Merge
df_data_w_coord = df_data_by_cell.merge(right=df_locations, left_on="skn", right_on="SKN")
df_data_w_coord.head()

,skn,year,month,data_in,SKN,Name,Lat_DD,Lon_DD
0,1.0,1948,1,3.2,1.0,KALAE,18.916176,-155.674994
1,1.0,1948,2,1.33,1.0,KALAE,18.916176,-155.674994
2,1.0,1948,3,2.31,1.0,KALAE,18.916176,-155.674994
3,1.0,1948,4,3.23,1.0,KALAE,18.916176,-155.674994
4,1.0,1948,5,1.14,1.0,KALAE,18.916176,-155.674994


## Load the cdf files

In [14]:
# ds = xr.open_dataset(f"{base_dir}/air.2m.mon.mean.regridded.nc")
lat_hawaii = [15, 17.5, 20, 22.5, 25]
lon_hawaii = np.array([-162.5, -160, -157.5, -155, -152.5]) + 360

ds_air2m = xr.open_dataset(f"{base_dir}/air.2m.mon.mean.regridded.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_air1000_500 = xr.open_dataset(f"{base_dir}/air.1000-500.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_hgt500 = xr.open_dataset(f"{base_dir}/hgt500.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_hgt1000 = xr.open_dataset(f"{base_dir}/hgt1000.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_omega500 = xr.open_dataset(f"{base_dir}/omega500.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_pottemp_1000_500 = xr.open_dataset(f"{base_dir}/pottmp.1000-500.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_pottemp_1000_850 = xr.open_dataset(f"{base_dir}/pottmp.1000-850.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_pwtr = xr.open_dataset(f"{base_dir}/pwtr.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_u700 = xr.open_dataset(f"{base_dir}/shum_x_uwnd.700.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_u925 = xr.open_dataset(f"{base_dir}/shum_x_uwnd.925.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_v700 = xr.open_dataset(f"{base_dir}/shum_x_vwnd.700.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_v950 = xr.open_dataset(f"{base_dir}/shum_x_vwnd.925.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_shum700 = xr.open_dataset(f"{base_dir}/shum700.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_shum925 = xr.open_dataset(f"{base_dir}/shum925.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_skt = xr.open_dataset(f"{base_dir}/skt.mon.mean.regridded.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_slp = xr.open_dataset(f"{base_dir}/slp.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]

datasets = [ # list of tuples. (dataset object, label, attribute string in ds)
    (ds_air2m, "air2m", "air"),
    (ds_air1000_500, "air1000_500", "air"),
    (ds_hgt500, "hgt500", "hgt"),
    (ds_hgt1000, "hgt1000", "hgt"),
    (ds_omega500, "omega500", "omega"),
    (ds_pottemp_1000_500, "pottemp1000-500", "pottmp"),
    (ds_pottemp_1000_850, "pottemp1000-850", "pottmp"),
    (ds_pwtr, "pr_wtr", "pr_wtr"),
    (ds_u700, "shum-uwnd-700", "shum"),
    (ds_u925, "shum-uwnd-925", "shum"),
    (ds_v700, "shum-vwnd-700", "shum"),
    (ds_v950, "shum-vwnd-950", "shum"),
    (ds_shum700, "shum700", "shum"),
    (ds_shum925, "shum925", "shum"),
    (ds_skt, "skt", "skt"),
    (ds_slp, "slp", "slp")
]
# combine all the cdf data

## Try new stuff

In [29]:
df_data['Year'].unique()

array([1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930,
       1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941,
       1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952,
       1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963,
       1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
       1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
       1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
       1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012])

In [43]:
new_data = []
for year in df_data["Year"].unique():
    if year < 1948: continue
    for month in [i + 1 for i in range(12)]:
        row = {}
        row["time"] = f"{year}-{month}-01"
        for ds, label, attribute in datasets:
            row[label] = ds.loc[dict(time=f"{year}-{month}-01")][attribute]
        new_data.append(row)

In [44]:
df = pd.DataFrame(new_data)
df.head()

,time,air2m,air1000_500,hgt500,hgt1000,omega500,pottemp1000-500,pottemp1000-850,pr_wtr,shum-uwnd-700,shum-uwnd-925,shum-vwnd-700,shum-vwnd-950,shum700,shum925,skt,slp
0,1948-1-01,[[<xarray.DataArray 'air' ()>\narray(297.59113...,[[[<xarray.DataArray 'air' ()>\narray(30.56000...,[[[<xarray.DataArray 'hgt' ()>\narray(5817.839...,[[[<xarray.DataArray 'hgt' ()>\narray(107.5806...,[[[<xarray.DataArray 'omega' ()>\narray(0.0249...,[[[<xarray.DataArray 'pottmp' ()>\narray(-27.6...,[[[<xarray.DataArray 'pottmp' ()>\narray(-3.40...,[[<xarray.DataArray 'pr_wtr' ()>\narray(33.237...,[[[<xarray.DataArray 'shum' ()>\narray(-3.2722...,[[[<xarray.DataArray 'shum' ()>\narray(-81.673...,[[[<xarray.DataArray 'shum' ()>\narray(0.51939...,[[[<xarray.DataArray 'shum' ()>\narray(-14.224...,[[[<xarray.DataArray 'shum' ()>\narray(2.597)\...,[[[<xarray.DataArray 'shum' ()>\narray(11.4710...,[[<xarray.DataArray 'skt' ()>\narray(25.64039)...,[[<xarray.DataArray 'slp' ()>\narray(1012.4068...
1,1948-2-01,[[<xarray.DataArray 'air' ()>\narray(296.67743...,[[[<xarray.DataArray 'air' ()>\narray(30.19999...,[[[<xarray.DataArray 'hgt' ()>\narray(5823.448...,[[[<xarray.DataArray 'hgt' ()>\narray(119.2413...,[[[<xarray.DataArray 'omega' ()>\narray(0.0489...,[[[<xarray.DataArray 'pottmp' ()>\narray(-28.0...,[[[<xarray.DataArray 'pottmp' ()>\narray(-4.22...,[[<xarray.DataArray 'pr_wtr' ()>\narray(27.417...,[[[<xarray.DataArray 'shum' ()>\narray(3.02897...,[[[<xarray.DataArray 'shum' ()>\narray(-55.066...,[[[<xarray.DataArray 'shum' ()>\narray(0.82366...,[[[<xarray.DataArray 'shum' ()>\narray(-7.3544...,[[[<xarray.DataArray 'shum' ()>\narray(2.657)\...,[[[<xarray.DataArray 'shum' ()>\narray(9.19299...,[[<xarray.DataArray 'skt' ()>\narray(23.903805...,[[<xarray.DataArray 'slp' ()>\narray(1013.7411...
2,1948-3-01,[[<xarray.DataArray 'air' ()>\narray(297.10855...,[[[<xarray.DataArray 'air' ()>\narray(31.63999...,[[[<xarray.DataArray 'hgt' ()>\narray(5822.096...,[[[<xarray.DataArray 'hgt' ()>\narray(119.3225...,[[[<xarray.DataArray 'omega' ()>\narray(-0.033...,[[[<xarray.DataArray 'pottmp' ()>\narray(-26.3...,[[[<xarray.DataArray 'pottmp' ()>\narray(-4.39...,[[<xarray.DataArray 'pr_wtr' ()>\narray(35.415...,[[[<xarray.DataArray 'shum' ()>\narray(2.51263...,[[[<xarray.DataArray 'shum' ()>\narray(-80.430...,[[[<xarray.DataArray 'shum' ()>\narray(8.59794...,[[[<xarray.DataArray 'shum' ()>\narray(4.69436...,[[[<xarray.DataArray 'shum' ()>\narray(3.92599...,[[[<xarray.DataArray 'shum' ()>\narray(10.4319...,[[<xarray.DataArray 'skt' ()>\narray(24.320047...,[[<xarray.DataArray 'slp' ()>\narray(1013.8130...
3,1948-4-01,[[<xarray.DataArray 'air' ()>\narray(297.90656...,[[[<xarray.DataArray 'air' ()>\narray(30.07)\n...,[[[<xarray.DataArray 'hgt' ()>\narray(5845.733...,[[[<xarray.DataArray 'hgt' ()>\narray(116.4666...,[[[<xarray.DataArray 'omega' ()>\narray(0.0359...,[[[<xarray.DataArray 'pottmp' ()>\narray(-28.4...,[[[<xarray.DataArray 'pottmp' ()>\narray(-4.08...,[[<xarray.DataArray 'pr_wtr' ()>\narray(35.977...,[[[<xarray.DataArray 'shum' ()>\narray(-5.4026...,[[[<xarray.DataArray 'shum' ()>\narray(-63.842...,[[[<xarray.DataArray 'shum' ()>\narray(-6.2063...,[[[<xarray.DataArray 'shum' ()>\narray(-6.6748...,[[[<xarray.DataArray 'shum' ()>\narray(4.465)\...,[[[<xarray.DataArray 'shum' ()>\narray(10.7660...,[[<xarray.DataArray 'skt' ()>\narray(25.467556...,[[<xarray.DataArray 'slp' ()>\narray(1013.4535...
4,1948-5-01,[[<xarray.DataArray 'air' ()>\narray(298.55862...,[[[<xarray.DataArray 'air' ()>\narray(29.91999...,[[[<xarray.DataArray 'hgt' ()>\narray(5860.935...,[[[<xarray.DataArray 'hgt' ()>\narray(119.9677...,[[[<xarray.DataArray 'omega' ()>\narray(0.0249...,[[[<xarray.DataArray 'pottmp' ()>\narray(-28.6...,[[[<xarray.DataArray 'pottmp' ()>\narray(-3.75...,[[<xarray.DataArray 'pr_wtr' ()>\narray(38.174...,[[[<xarray.DataArray 'shum' ()>\narray(-16.938...,[[[<xarray.DataArray 'shum' ()>\narray(-130.43...,[[[<xarray.DataArray 'shum' ()>\narray(-2.5948...,[[[<xarray.DataArray 's

In [50]:
df_data_w_coord.head()

,skn,year,month,data_in,SKN,Name,Lat_DD,Lon_DD
0,1.0,1948,1,3.2,1.0,KALAE,18.916176,-155.674994
1,1.0,1948,2,1.33,1.0,KALAE,18.916176,-155.674994
2,1.0,1948,3,2.31,1.0,KALAE,18.916176,-155.674994
3,1.0,1948,4,3.23,1.0,KALAE,18.916176,-155.674994
4,1.0,1948,5,1.14,1.0,KALAE,18.916176,-155.674994


In [52]:
df_data_w_coord["time"] = df_data_w_coord.apply(lambda x: f"{x.year}-{x.month}-01", axis=1)
df_data_w_coord

,skn,year,month,data_in,SKN,Name,Lat_DD,Lon_DD,time
0,1.0,1948,1,3.2,1.0,KALAE,18.916176,-155.674994,1948-1-01
1,1.0,1948,2,1.33,1.0,KALAE,18.916176,-155.674994,1948-2-01
2,1.0,1948,3,2.31,1.0,KALAE,18.916176,-155.674994,1948-3-01
3,1.0,1948,4,3.23,1.0,KALAE,18.916176,-155.674994,1948-4-01
4,1.0,1948,5,1.14,1.0,KALAE,18.916176,-155.674994,1948-5-01
...,...,...,...,...,...,...,...,...,...
865556,1147.0,1973,11,13.58,1147.0,FLD 960-MOLOAA,22.174441,-159.336092,1973-11-01
865557,1147.0,1973,12,8.23,1147.0,FLD 960-MOLOAA,22.174441,-159.336092,1973-12-01
865558,1147.0,1974,1,7.51,1147.0,FLD 960-MOLOAA,22.174441,-159.336092,1974-1-01
865559,1147.0,1974,2,2.31,1147.0,FLD 960-MOLOAA,22.174441,-159.336092,1974-2-01


In [64]:
df_complete = df_data_w_coord.merge(right=df, left_on="time", right_on="time")

In [66]:
df_complete.head()

,skn,year,month,data_in,SKN,Name,Lat_DD,Lon_DD,time,air2m,...,pottemp1000-850,pr_wtr,shum-uwnd-700,shum-uwnd-925,shum-vwnd-700,shum-vwnd-950,shum700,shum925,skt,slp
0,1.00,1948,1,3.2,1.00,KALAE,18.916176,-155.674994,1948-1-01,[[<xarray.DataArray 'air' ()>\narray(297.59113...,...,[[[<xarray.DataArray 'pottmp' ()>\narray(-3.40...,[[<xarray.DataArray 'pr_wtr' ()>\narray(33.237...,[[[<xarray.DataArray 'shum' ()>\narray(-3.2722...,[[[<xarray.DataArray 'shum' ()>\narray(-81.673...,[[[<xarray.DataArray 'shum' ()>\narray(0.51939...,[[[<xarray.DataArray 'shum' ()>\narray(-14.224...,[[[<xarray.DataArray 'shum' ()>\narray(2.597)\...,[[[<xarray.DataArray 'shum' ()>\narray(11.4710...,[[<xarray.DataArray 'skt' ()>\narray(25.64039)...,[[<xarray.DataArray 'slp' ()>\narray(1012.4068...
1,2.00,1948,1,5.95,2.00,MANUKA,19.108660,-155.825545,1948-1-01,[[<xarray.DataArray 'air' ()>\narray(297.59113...,...,[[[<xarray.DataArray 'pottmp' ()>\narray(-3.40...,[[<xarray.DataArray 'pr_wtr' ()>\narray(33.237...,[[[<xarray.DataArray 'shum' ()>\narray(-3.2722...,[[[<xarray.DataArray 'shum' ()>\narray(-81.673...,[[[<xarray.DataArray 'shum' ()>\narray(0.51939...,[[[<xarray.DataArray 'shum' ()>\narray(-14.224...,[[[<xarray.DataArray 'shum' ()>\narray(2.597)\...,[[[<xarray.DataArray 'shum' ()>\narray(11.4710...,[[<xarray.DataArray 'skt' ()>\narray(25.64039)...,[[<xarray.DataArray 'slp' ()>\narray(1012.4068...
2,2.20,1948,1,11.5,2.20,KAHUKU SHED 3,19.164740,-155.682280,1948-1-01,[[<xarray.DataArray 'air' ()>\narray(297.59113...,...,[[[<xarray.DataArray 'pottmp' ()>\narray(-3.40...,[[<xarray.DataArray 'pr_wtr' ()>\narray(33.237...,[[[<xarray.DataArray 'shum' ()>\narray(-3.2722...,[[[<xarray.DataArray 'shum' ()>\narray(-81.673...,[[[<xarray.DataArray 'shum' ()>\narray(0.51939...,[[[<xarray.DataArray 'shum' ()>\narray(-14.224...,[[[<xarray.DataArray 'shum' ()>\narray(2.597)\...,[[[<xarray.DataArray 'shum' ()>\narray(11.4710...,[[<xarray.DataArray 'skt' ()>\narray(25.64039)...,[[<xarray.DataArray 'slp' ()>\narray(1012.4068...
3,2.25,1948,1,5.515941,2.25,RESERVOIR (2940),19.160603,-155.822488,1948-1-01,[[<xarray.DataArray 'air' ()>\narray(297.59113...,...,[[[<xarray.DataArray 'pottmp' ()>\narray(-3.40...,[[<xarray.DataArray 'pr_wtr' ()>\narray(33.237...,[[[<xarray.DataArray 'shum' ()>\narray(-3.2722...,[[[<xarray.DataArray 'shum' ()>\narray(-81.673...,[[[<xarray.DataArray 'shum' ()>\narray(0.51939...,[[[<xarray.DataArray 'shum' ()>\narray(-14.224...,[[[<xarray.DataArray 'shum' ()>\narray(2.597)\...,[[[<xarray.DataArray 'shum' ()>\narray(11.4710...,[[<xarray.DataArray 'skt' ()>\narray(25.64039)...,[[<xarray.DataArray 'slp' ()>\narray(1012.4068...
4,2.26,1948,1,4.310617,2.26,CASTLE,19.225323,-155.778876,1948-1-01,[[<xarray.DataArray 'air' ()>\narray(297.59113...,...,[[[<xarray.DataArray 'pottmp' ()>\narray(-3.40...,[[<xarray.DataArray 'pr_wtr' ()>\narray(33.237...,[[[<xarray.DataArray 'shum' ()>\narray(-3.2722...,[[[<xarray.DataArray 'shum' ()>\narray(-81.673...,[[[<xarray.DataArray 'shum' ()>\narray(0.51939...,[[[<xarray.DataArray 'shum' ()>\narray(-14.224...,[[[<xarray.DataArray 'shum' ()>\narray(2.597)\...,[[[<xarray.DataArray 'shum' ()>\narray(11.4710...,[[<xarray.DataArray 'skt' ()>\narray(25.64039)...,[[<xarray.DataArray 'slp' ()>\narray(1012.4068...


In [69]:
df_complete.iloc[1002]["shum925"]

<xarray.DataArray 'shum' (level: 1, lat: 5, lon: 5)>
array([[[11.471001, 11.316999, 10.889999, 10.625999, 10.604   ],
        [10.851999, 10.669001, 10.110001,  9.851999, 10.087   ],
        [ 9.972   ,  9.947001,  9.923   ,  9.869999,  9.778999],
        [ 9.235001,  9.209999,  9.313   ,  9.396999,  9.289999],
        [ 8.246   ,  8.576   ,  8.778   ,  8.764999,  8.750999]]],
      dtype=float32)
Coordinates:
  * lat      (lat) float32 15.0 17.5 20.0 22.5 25.0
  * level    (level) float32 925.0
  * lon      (lon) float32 197.5 200.0 202.5 205.0 207.5
    time     datetime64[ns] 1948-01-01
Attributes:
    long_name:     Monthly Mean of Specific Humidity
    units:         grams/kg
    precision:     3
    var_desc:      Specific Humidity
    level_desc:    Multiple levels
    statistic:     Mean
    parent_stat:   Other
    valid_range:   [-9.999999e-02  1.004300e+02]
    dataset:       NCEP Reanalysis Derived Products
    actual_range:  [6.9999695e-03 2.1206001e+01]

In [72]:
ds_shum925.loc[dict(time="1948-01-01")]["shum"]

<xarray.DataArray 'shum' (level: 1, lat: 5, lon: 5)>
array([[[11.471001, 11.316999, 10.889999, 10.625999, 10.604   ],
        [10.851999, 10.669001, 10.110001,  9.851999, 10.087   ],
        [ 9.972   ,  9.947001,  9.923   ,  9.869999,  9.778999],
        [ 9.235001,  9.209999,  9.313   ,  9.396999,  9.289999],
        [ 8.246   ,  8.576   ,  8.778   ,  8.764999,  8.750999]]],
      dtype=float32)
Coordinates:
  * lat      (lat) float32 15.0 17.5 20.0 22.5 25.0
  * level    (level) float32 925.0
  * lon      (lon) float32 197.5 200.0 202.5 205.0 207.5
    time     datetime64[ns] 1948-01-01
Attributes:
    long_name:     Monthly Mean of Specific Humidity
    units:         grams/kg
    precision:     3
    var_desc:      Specific Humidity
    level_desc:    Multiple levels
    statistic:     Mean
    parent_stat:   Other
    valid_range:   [-9.999999e-02  1.004300e+02]
    dataset:       NCEP Reanalysis Derived Products
    actual_range:  [6.9999695e-03 2.1206001e+01]

In [13]:
start = time()
df_data_xarray = pd.DataFrame()

for dataset in datasets:
    array = dataset[0]
    label = dataset[1]
    df_data_xarray[label] = df_data_w_coord[:10].apply(lambda x: array.loc[dict(time=f"{x['year']}-{x['month']}-01")].to_array(), axis=1)
end = time()
print(end-start)

0.48003602027893066


In [25]:
df_data_xarray.head()

,air2m,air1000_500,hgt500,hgt1000,omega500,pottemp1000-500,pottemp1000-850,pr_wtr,shum-uwnd-700,shum-uwnd-925,shum-vwnd-700,shum-vwnd-950,shum700,shum925,skt,slp
0,"[[[<xarray.DataArray ()>\narray(297.59113, dty...",[[[<xarray.DataArray (lon: 5)>\narray([30.5600...,[[[<xarray.DataArray (lon: 5)>\narray([5817.83...,[[[<xarray.DataArray (lon: 5)>\narray([107.580...,[[[<xarray.DataArray (lon: 5)>\narray([0.02499...,[[[<xarray.DataArray (lon: 5)>\narray([-27.681...,[[[<xarray.DataArray (lon: 5)>\narray([-3.4075...,"[[[<xarray.DataArray ()>\narray(33.23709, dtyp...",[[[<xarray.DataArray (lon: 5)>\narray([-3.2722...,[[[<xarray.DataArray (lon: 5)>\narray([-81.673...,[[[<xarray.DataArray (lon: 5)>\narray([ 0.5193...,[[[<xarray.DataArray (lon: 5)>\narray([-14.224...,[[[<xarray.DataArray (lon: 5)>\narray([2.59700...,[[[<xarray.DataArray (lon: 5)>\narray([11.4710...,"[[[<xarray.DataArray ()>\narray(25.64039, dtyp...","[[[<xarray.DataArray ()>\narray(1012.4068, dty..."
1,"[[[<xarray.DataArray ()>\narray(296.67743, dty...",[[[<xarray.DataArray (lon: 5)>\narray([30.1999...,[[[<xarray.DataArray (lon: 5)>\narray([5823.44...,[[[<xarray.DataArray (lon: 5)>\narray([119.241...,[[[<xarray.DataArray (lon: 5)>\narray([ 0.0489...,[[[<xarray.DataArray (lon: 5)>\narray([-28.010...,[[[<xarray.DataArray (lon: 5)>\narray([-4.2215...,"[[[<xarray.DataArray ()>\narray(27.41724, dtyp...",[[[<xarray.DataArray (lon: 5)>\narray([3.02897...,[[[<xarray.DataArray (lon: 5)>\narray([-55.066...,[[[<xarray.DataArray (lon: 5)>\narray([ 0.8236...,[[[<xarray.DataArray (lon: 5)>\narray([-7.3544...,[[[<xarray.DataArray (lon: 5)>\narray([2.65699...,[[[<xarray.DataArray (lon: 5)>\narray([ 9.1929...,"[[[<xarray.DataArray ()>\narray(23.903805, dty...","[[[<xarray.DataArray ()>\narray(1013.7411, dty..."
2,"[[[<xarray.DataArray ()>\narray(297.10855, dty...",[[[<xarray.DataArray (lon: 5)>\narray([31.64 ...,[[[<xarray.DataArray (lon: 5)>\narray([5822.09...,[[[<xarray.DataArray (lon: 5)>\narray([119.322...,[[[<xarray.DataArray (lon: 5)>\narray([-0.0330...,[[[<xarray.DataArray (lon: 5)>\narray([-26.317...,[[[<xarray.DataArray (lon: 5)>\narray([-4.3960...,"[[[<xarray.DataArray ()>\narray(35.41548, dtyp...",[[[<xarray.DataArray (lon: 5)>\narray([2.51263...,[[[<xarray.DataArray (lon: 5)>\narray([-80.430...,[[[<xarray.DataArray (lon: 5)>\narray([ 8.5979...,[[[<xarray.DataArray (lon: 5)>\narray([ 4.6943...,[[[<xarray.DataArray (lon: 5)>\narray([3.92599...,[[[<xarray.DataArray (lon: 5)>\narray([10.4319...,"[[[<xarray.DataArray ()>\narray(24.320047, dty...","[[[<xarray.DataArray ()>\narray(1013.81305, dt..."
3,"[[[<xarray.DataArray ()>\narray(297.90656, dty...",[[[<xarray.DataArray (lon: 5)>\narray([30.07 ...,[[[<xarray.DataArray (lon: 5)>\narray([5845.73...,[[[<xarray.DataArray (lon: 5)>\narray([116.466...,[[[<xarray.DataArray (lon: 5)>\narray([0.03599...,[[[<xarray.DataArray (lon: 5)>\narray([-28.403...,[[[<xarray.DataArray (lon: 5)>\narray([-4.0877...,"[[[<xarray.DataArray ()>\narray(35.977665, dty...",[[[<xarray.DataArray (lon: 5)>\narray([-5.4026...,[[[<xarray.DataArray (lon: 5)>\narray([-63.842...,[[[<xarray.DataArray (lon: 5)>\narray([-6.2063...,[[[<xarray.DataArray (lon: 5)>\narray([ -6.674...,[[[<xarray.DataArray (lon: 5)>\narray([4.465 ...,[[[<xarray.DataArray (lon: 5)>\narray([10.7660...,"[[[<xarray.DataArray ()>\narray(25.467556, dty...","[[[<xarray.DataArray ()>\narray(1013.4535, dty..."
4,"[[[<xarray.DataArray ()>\narray(298.55862, dty...",[[[<xarray.DataArray (lon: 5)>\narray([29.9199...,[[[<xarray.DataArray (lon: 5)>\narray([5860.93...,[[[<xarray.DataArray (lon: 5)>\narray([119.967...,[[[<xarray.DataArray (lon: 5)>\narray([0.02499...,[[[<xarray.DataArray (lon: 5)>\narray([-28.676...,[[[<xarray.DataArray (lon: 5)>\narray([-3.7547...,"[[[<xarray.DataArray ()>\narray(38.174515, dty...",[[[<xarray.DataArray (lon: 5)>\narray([-16.938...,[[[<xarray.DataArray (lon: 5)>\narray([-130.43...,[[[<xarray.DataArray (lon: 5)>\narray([-2.5948...,[[[<xarray.DataArray (lon: 5)>\narray([-16.27

In [26]:
df_data_xarray.to_csv(f"{base_dir}/temp.csv")

In [22]:
df_testing = pd.read_csv(f"{base_dir}/temp.csv")

In [23]:
df_testing

,Unnamed: 0,air2m,air1000_500,hgt500,hgt1000,omega500,pottemp1000-500,pottemp1000-850,pr_wtr,shum-uwnd-700,shum-uwnd-925,shum-vwnd-700,shum-vwnd-950,shum700,shum925,skt,slp
0,0,"<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, lat: 5, lon: 5..."
1,1,"<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, lat: 5, lon: 5..."
2,2,"<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, lat: 5, lon: 5..."
3,3,"<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, lat: 5, lon: 5..."
4,4,"<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.DataArray (variable: 1, lat: 5, lon: 5...","<xarray.DataArray (variable: 1, level: 1, lat:...","<xarray.Da